# Table of Contents
 <p><div class="lev1"><a href="#overview-and-getting-started"><span class="toc-item-num">1&nbsp;&nbsp;</span>overview and getting started</a></div><div class="lev2"><a href="#Introduction"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Introduction</a></div><div class="lev2"><a href="#Architecture-overview"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Architecture overview</a></div><div class="lev3"><a href="#IPython-engine"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>IPython engine</a></div><div class="lev3"><a href="#IPython-controller"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>IPython controller</a></div><div class="lev3"><a href="#The-Hub"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>The Hub</a></div><div class="lev3"><a href="#Schedulers"><span class="toc-item-num">1.2.4&nbsp;&nbsp;</span>Schedulers</a></div><div class="lev2"><a href="#IPython-client-and-views"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>IPython client and views</a></div><div class="lev2"><a href="#Getting-started:-Starting-the-IPython-controller-and-engines"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Getting started: Starting the IPython controller and engines</a></div><div class="lev2"><a href="#Configuring-an-IPython-cluster"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Configuring an IPython cluster</a></div><div class="lev1"><a href="#IPython’s-Direct-interface"><span class="toc-item-num">2&nbsp;&nbsp;</span>IPython’s Direct interface</a></div>

# overview and getting started

## Introduction

This section gives an overview of IPython’s architecture for parallel and distributed computing. This architecture abstracts out parallelism in a very general way, which enables IPython to support many different styles of parallelism including:

- Single program, multiple data (SPMD) parallelism.
- Multiple program, multiple data (MPMD) parallelism.
- Message passing using MPI.
- Task farming.
- Data parallel.
- Combinations of these approaches.
- Custom user defined approaches.

Most importantly, IPython enables all types of parallel applications to be developed, executed, debugged and 
monitored interactively. Hence, the I in IPython. The following are some example usage cases for IPython:

- Quickly parallelize algorithms that are embarrassingly parallel using a number of simple approaches. Many simple things can be parallelized interactively in one or two lines of code.
- Steer traditional MPI applications on a supercomputer from an IPython session on your laptop.
- Analyze and visualize large datasets (that could be remote and/or distributed) interactively using IPython and tools like matplotlib/TVTK.
- Develop, test and debug new parallel algorithms (that may use MPI) interactively.
- Tie together multiple MPI jobs running on different systems into one giant distributed and parallel system.
- Start a parallel job on your cluster and then have a remote collaborator connect to it and pull back data into their local IPython session for plotting and analysis.
- Run a set of tasks on a set of CPUs using dynamic load balancing.

## Architecture overview

<img src="wideView.png">

The IPython architecture consists of four components:

- The IPython engine.
- The IPython hub.
- The IPython schedulers.
- The controller client.

These components live in the ```IPython.parallel``` package and are installed with IPython.

### IPython engine

The IPython engine is a Python instance that takes Python commands over a network connection. Eventually, the IPython engine will be a full IPython interpreter, but for now, it is a regular Python interpreter. The engine can also handle incoming and outgoing Python objects sent over a network connection. When multiple engines are started, parallel and distributed computing becomes possible. An important feature of an IPython engine is that it blocks while user code is being executed. Read on for how the IPython controller solves this problem to expose a clean asynchronous API to the user.

### IPython controller

he IPython controller processes provide an interface for working with a set of engines. At a general level, the controller is a collection of processes to which IPython engines and clients can connect. The controller is composed of a **Hub** and a collection of **Schedulers**. These Schedulers are typically run in separate processes but on the same machine as the Hub, but can be run anywhere from local threads or on remote machines.

The controller also provides a single point of contact for users who wish to utilize the engines connected to the controller. There are different ways of working with a controller. In IPython, all of these models are implemented via the **View.apply()** method, after constructing **View** objects to represent subsets of engines. The two primary models for interacting with engines are:

- A **Direct** interface, where engines are addressed explicitly.
- A **LoadBalanced** interface, where the Scheduler is trusted with assigning work to appropriate engines.
Advanced users can readily extend the View models to enable other styles of parallelism.

Note:
```
A single controller and set of engines can be used with multiple models simultaneously. This opens the door for lots of interesting things.
```

### The Hub

The center of an IPython cluster is the Hub. This is the process that keeps track of engine connections, schedulers, clients, as well as all task requests and results. The primary role of the Hub is to facilitate queries of the cluster state, and minimize the necessary information required to establish the many connections involved in connecting new clients and engines.


### Schedulers

All actions that can be performed on the engine go through a Scheduler. While the engines themselves block when user code is run, the schedulers hide that from the user to provide a fully asynchronous interface to a set of engines.

## IPython client and views

There is one primary object, the **Client**, for connecting to a cluster. For each execution model, there is a corresponding **View**. These views allow users to interact with a set of engines through the interface. Here are the two default views:

- The **DirectView** class for explicit addressing.
- The **LoadBalancedView** class for destination-agnostic scheduling.

## Getting started: Starting the IPython controller and engines

You will need to start the IPython controller and four IPython engines. The simplest way of doing this is to use the ipcluster command.

There are many more options available, but they are outside the scope of this introduction.

The ipcluster command provides a simple way of starting a controller and engines in the following situations:

- When the controller and engines are all run on localhost. This is useful for testing or running on a multicore computer.
- When engines are started using the mpiexec command that comes with most MPI [MPI] implementations
- When engines are started using the PBS [PBS] batch system (or other qsub systems, such as SGE).
- When the controller is started on localhost and the engines are started on remote nodes using ssh.
- When engines are started using the Windows HPC Server batch system.

**Note:**
```
Currently ipcluster requires that the IPYTHONDIR/profile_<name>/security directory live on a shared filesystem that is seen by both the controller and engines. If you don’t have a shared file system you will need to use ipcontroller and ipengine directly.
```

Under the hood, ipcluster just uses ipcontroller and ipengine to perform the steps described above.

The simplest way to use ipcluster requires no configuration, and will launch a controller and a number of engines on the local machine. If you are using anaconda, it is warmly advised to first source your conda environment. In this way, issues with libraries are avoided. To start one controller and 4 engines on localhost, just do:

```
$ source activate YOUR_ENV_NAME
$ ipcluster start --n=4
```

## Configuring an IPython cluster

Cluster configurations are stored as profiles. You can create a new profile with:

```
$ ipython profile create --parallel --profile=myprofile
```

This will create the directory ```IPYTHONDIR/profile_myprofile```, and populate it with the default configuration files for the three IPython cluster commands. Once you edit those files, you can continue to call ipcluster/ipcontroller/ipengine with no arguments beyond ```profile=myprofile```, and any configuration will be maintained.

There is no limit to the number of profiles you can have, so you can maintain a profile for each of your common use cases. The default profile will be used whenever the profile argument is not specified, so edit ```IPYTHONDIR/profile_default/*_config.py``` to represent your most common use case.

The configuration files are loaded with commented-out settings and explanations, which should cover most of the available possibilities.

Using ipcluster with your profile:

```
$ source activate YOUR_ENV_NAME
$ ipcluster start --n=4 --profile='YOUR_IPYTHON_PROFILE'
```